# Data processing, sampling and KwARG

This notebook provides commands used in processing the data, obtaining the samples and running KwARG. The commands can be run on most Unix-based systems via command line (to re-run a particular command, delete the '!' prefixing each line below).

External tools used:

[MAFFT](https://mafft.cbrc.jp/alignment/software/)

[SeqKit](https://bioinf.shenwei.me/seqkit/)

# Data processing : England

### Source

The data was downloaded from [GISAID](https://www.gisaid.org/) on 20 December 2020, filtering for sequences:

- collected in November 2020 in England;
- marked as complete (>29,900 nucleotides) and excluding low coverage sequences (>5% ambiguous nucleotides);
- labelled as clade GR;

giving 4,517 sequences in total. The downloaded files are named:

- `Eng_data.fasta` (the sequences in fasta format)
- `Eng_sequencing_info.tsv` (sequencing metadata)

The data is not shared here as per GISAID's terms of use, but the GISAID accession IDs are provided in the file Eng_id.txt, which can be used to recreate the dataset used in our analysis:

In [1]:
!grep ">" Eng_data.fasta | awk -F"|" '{ print $2 }' > Eng_id.txt;
!head Eng_id.txt

EPI_ISL_637284
EPI_ISL_642611
EPI_ISL_642613
EPI_ISL_642618
EPI_ISL_642621
EPI_ISL_642624
EPI_ISL_642627
EPI_ISL_642629
EPI_ISL_642630
EPI_ISL_642631


### Alignment

The sequences in each dataset were aligned using MAFFT to the reference sequence in `ref_seq.fasta` (GISAID accession EPI_ISL_402125, GenBank ID MN908947.3):
```sh
mafft --auto --thread -1 --keeplength --quiet --mapout --preservecase --addfragments Eng_data.fasta ref_seq.fasta > Eng_alignment.fasta
```

All symbols other than 'A, C, T, G' are replaced with 'N':

In [2]:
!sed '/^>/! s/[^actgACTG]/N/g' Eng_alignment.fasta > Eng_alignment_cleaned.fasta

### Splitting by strain

The IDs corresponding to lineage B.1.1.7 are extracted from the sequencing metadata:

In [3]:
!grep "B.1.1.7	" Eng_sequencing_info.tsv > Eng_newstrain.txt;
!awk '{print $1"|" $2 "|" $3}' Eng_newstrain.txt > Eng_newstrain_id.txt;
!rm Eng_newstrain.txt

The alignment is split into two files by lineage:

In [4]:
!seqkit grep -n -f Eng_newstrain_id.txt Eng_alignment_cleaned.fasta > Eng_newstrain.fasta;
!seqkit grep -n -v -f Eng_newstrain_id.txt Eng_alignment_cleaned.fasta > Eng_oldstrain.fasta

To avoid having identical sequences in the sample, all sequences which are identical are removed:

In [5]:
!seqkit rmdup -DEng_olddeleted.txt -s Eng_oldstrain.fasta > Eng_oldstrain_nondup.fasta;
!seqkit rmdup -DEng_newdeleted.txt -s Eng_newstrain.fasta > Eng_newstrain_nondup.fasta;
!rm Eng_newstrain.fasta;
!rm Eng_oldstrain.fasta

[INFO] 465 duplicated records removed
[INFO] 468 duplicated records removed


In [6]:
!seqkit stats Eng_newstrain_nondup.fasta;
!seqkit stats Eng_oldstrain_nondup.fasta

file                        format  type  num_seqs     sum_len  min_len  avg_len  max_len
Eng_newstrain_nondup.fasta  FASTA   DNA        934  27,929,402   29,903   29,903   29,903
file                        format  type  num_seqs     sum_len  min_len  avg_len  max_len
Eng_oldstrain_nondup.fasta  FASTA   DNA      2,651  79,272,853   29,903   29,903   29,903


### Sample generation

40 sequences from each dataset are selected using SeqKit:

In [7]:
!cat ref_seq.fasta > Eng_sample.fasta;
!seqkit sample -s 62143677 -p 0.1 Eng_oldstrain_nondup.fasta | seqkit shuffle -s 73487948 | seqkit head -n 40 >> Eng_sample.fasta;
!seqkit sample -s 92873461 -p 0.1 Eng_newstrain_nondup.fasta | seqkit shuffle -s 48182303 | seqkit head -n 40 >> Eng_sample.fasta;

[INFO] read sequences ...
[INFO] sample by proportion
[INFO] 247 sequences outputted
[INFO] 247 sequences loaded
[INFO] shuffle ...
[INFO] output ...
[INFO] read sequences ...
[INFO] sample by proportion
[INFO] 83 sequences outputted
[INFO] 83 sequences loaded
[INFO] shuffle ...
[INFO] output ...


The IDs of sequences in the sample are found in sample_ids.txt:

In [8]:
!grep ">" Eng_sample.fasta | awk -F"|" '{ print $2 }' > Eng_sample_id.txt;
!head Eng_sample_id.txt


EPI_ISL_662468
EPI_ISL_664402
EPI_ISL_702752
EPI_ISL_650455
EPI_ISL_667977
EPI_ISL_642566
EPI_ISL_661404
EPI_ISL_679726
EPI_ISL_654967


To check the size of the resulting sample:

In [9]:
!seqkit stats Eng_sample.fasta

file              format  type  num_seqs    sum_len  min_len  avg_len  max_len
Eng_sample.fasta  FASTA   DNA         81  2,422,143   29,903   29,903   29,903


This consists of 80 sampled sequences plus the reference.

### Masking problematic sites

The R script `Eng_find_multiallelic.R` is used to extract positions of the SNPs in the data, identify and mask multi-allelic sites, and mask all sites flagged as problematic by [De Maio et al (2020)](https://virological.org/t/issues-with-sars-cov-2-sequencing-data/473) (updated to 13 November 2020). The list of problematic sites is downloaded from [here](https://github.com/W-L/ProblematicSites_SARS-CoV2).

In [10]:
!awk '/^>/ { print (NR==1 ? "" : RS) $0; next } { printf "%s", $0 } END { printf RS }' Eng_sample.fasta > Eng_sample_temp.fasta;
!sed 's/./& /g' < Eng_sample_temp.fasta > Eng_sample_test.fasta;
!rm Eng_sample_temp.fasta;
!Rscript Eng_find_multiallelic.R Eng_sample_test.fasta Eng_sample_masked.fasta Eng_sample_positions.fasta

[1] SNP sites: 363
[1] Plus masked: 7
[1] Plus multi-allelic: 3
[1] SNP sites:
  [1]   145   204   213   219   241   337   406   505   509   583   596   832
 [13]   872   913   936  1126  1149  1163  1191  1210  1288  1344  1457  1471
 [25]  1537  1627  1681  1820  1883  1886  1912  1919  1942  2110  2232  2342
 [37]  2380  2407  2445  2453  2494  2571  2716  2722  2880  3004  3037  3117
 [49]  3176  3251  3256  3259  3261  3267  3330  3331  3373  3587  3602  3638
 [61]  3686  3695  3713  3827  3873  3880  3896  3959  4002  4009  4021  4255
 [73]  4291  4303  4345  4573  4763  4890  4964  5100  5128  5139  5250  5278
 [85]  5388  5392  5575  5622  5672  5986  6160  6190  6317  6633  6730  6781
 [97]  6790  6868  6883  6926  6936  6954  7589  7732  7783  7919  8072  8084
[109]  8118  8353  8476  8565  8603  8683  8917  9051  9072  9130  9207  9289
[121]  9433  9484  9611  9623  9802  9836  9857 10027 10097 10189 10196 10265
[133] 10369 10448 10789 10969 10992 10994 11230 11401 11447 115

There are 373 variable sites, of which 3 are multi-allelic, and a further 7 are masked as problematic. The output file `Eng_sample_masked.fasta` contains the masked multiple alignment; the file `Eng_positions.txt` contains a list of SNP positions.

### Labelling

The sequences are given sequential labels (to make the ARGs easier to view):

In [11]:
!seqkit head -n 1 Eng_sample_masked.fasta > Eng_sample_masked_id.fasta;
!seqkit range -r 2:41 Eng_sample_masked.fasta > f1.f;
!seqkit range -r 42:81 Eng_sample_masked.fasta > f2.f;
!awk '/^>/{print ">O" ++i; next}{print}' < f1.f >> Eng_sample_masked_id.fasta;
!awk '/^>/{print ">N" ++i; next}{print}' < f2.f >> Eng_sample_masked_id.fasta;
!rm f1.f;
!rm f2.f

# Data processing: South Africa

### Source

The data was downloded from [GISAID](https://www.gisaid.org/) on 28 December 2020, filtering for sequences:

- collected in November 2020 in South Africa;
- marked as complete (>29,900 nucleotides) and excluding low coverage sequences (>5% ambiguous nucleotides);

giving 326 sequences in total. The downloaded files are named:

- `SA_data.fasta` (the sequences in fasta format)
- `SA_sequencing_info.tsv` (sequencing metadata)

Filtering only for sequences labelled as belonging to strain 501Y.V2, the sequencing metadata was downloaded in file

- `SA_newstrain_sequencing_info.tsv`

to enable filtering by strain.

The data is not shared here as per GISAID's terms of use, but the GISAID accession IDs are provided in the file SA_id.txt, which can be used to recreate the dataset used in our analysis:

In [12]:
!grep ">" SA_data.fasta | awk -F"|" '{ print $2 }' > SA_id.txt;
!head SA_id.txt

EPI_ISL_660159
EPI_ISL_660160
EPI_ISL_660161
EPI_ISL_660162
EPI_ISL_660163
EPI_ISL_660164
EPI_ISL_660221
EPI_ISL_660222
EPI_ISL_660225
EPI_ISL_660228


### Alignment

The sequences in each dataset were aligned using MAFFT as above and all symbols other than 'A, C, T, G' were replaced with 'N', as above. The resulting multiple alignment was stored in `SA_alignment_cleaned.fasta`.

Sequences labelled as having long stretches of ambiguous nucleotides were removed:

In [13]:
!grep "Stretches" SA_sequencing_info.tsv | awk '{print $1 " " $2 "|" $3 "|" $4}' > Ns_id.txt;
!sed -i .bak "s/\ /_/g" Ns_id.txt;
!sed -i .bak "s/\ /_/g" SA_alignment_cleaned.fasta;
!seqkit grep -n -v -f Ns_id.txt SA_alignment_cleaned.fasta > SA_alignment_good.fasta

To check the size of the resulting dataset:

In [14]:
!seqkit stats SA_alignment_good.fasta

file                     format  type  num_seqs    sum_len  min_len  avg_len  max_len
SA_alignment_good.fasta  FASTA   DNA        279  8,342,937   29,903   29,903   29,903


### Splitting by strain

The sequencing metadata file `SA_newstrain_sequencing_info.txt` is used to extract the sequences which are labelled as belonging to strain 501Y.V2.

In [15]:
!awk '{print $1 " " $2 "|" $3 "|" $4}' SA_newstrain_sequencing_info.tsv | grep "hCoV" > SA_newstrain_id.txt;
!sed -i .bak "s/\ /_/g" SA_newstrain_id.txt

In [16]:
!seqkit grep -n -f SA_newstrain_id.txt SA_alignment_good.fasta > SA_newstrain.fasta;
!seqkit grep -n -v -f SA_newstrain_id.txt SA_alignment_good.fasta > SA_oldstrain.fasta

In [17]:
!seqkit rmdup -DSA_olddeleted.txt -s SA_oldstrain.fasta > SA_oldstrain_nondup.fasta;
!seqkit rmdup -DSA_newdeleted.txt -s SA_newstrain.fasta > SA_newstrain_nondup.fasta;
!rm SA_newstrain.fasta;
!rm SA_oldstrain.fasta

[INFO] 0 duplicated records removed
[INFO] 0 duplicated records removed


In [18]:
!seqkit stats SA_oldstrain_nondup.fasta;
!seqkit stats SA_newstrain_nondup.fasta

file                       format  type  num_seqs    sum_len  min_len  avg_len  max_len
SA_oldstrain_nondup.fasta  FASTA   DNA        102  3,050,106   29,903   29,903   29,903
file                       format  type  num_seqs    sum_len  min_len  avg_len  max_len
SA_newstrain_nondup.fasta  FASTA   DNA        177  5,292,831   29,903   29,903   29,903


### Sample generation

The reference sequence is deleted from `SA_oldstrain_nondup.fasta` prior to running the following:

In [19]:
!cat ref_seq.fasta > SA_sample.fasta;
!seqkit sample -s 43984291 -p 0.6 SA_oldstrain_nondup.fasta | seqkit shuffle -s 92834717 | seqkit head -n 25 >> SA_sample.fasta;
!seqkit sample -s 23849817 -p 0.6 SA_newstrain_nondup.fasta | seqkit shuffle -s 34876261 | seqkit head -n 25 >> SA_sample.fasta;

[INFO] sample by proportion
[INFO] read sequences ...
[INFO] 62 sequences outputted
[INFO] 62 sequences loaded
[INFO] shuffle ...
[INFO] output ...
[INFO] sample by proportion
[INFO] read sequences ...
[INFO] 113 sequences outputted
[INFO] 113 sequences loaded
[INFO] shuffle ...
[INFO] output ...


The accession numbers of the sampled sequences are given in `SA_sample_id.txt`:

In [20]:
!grep ">" SA_sample.fasta | awk -F"|" '{ print $2 }' > SA_sample_id.txt;
!head SA_sample_id.txt


EPI_ISL_660225
EPI_ISL_660257
EPI_ISL_736993
EPI_ISL_660643
EPI_ISL_660229
EPI_ISL_736985
EPI_ISL_736926
EPI_ISL_696462
EPI_ISL_660655


### Masking problematic sites

This is done using the same method as for the England dataset, with the script modified to mask additional sites as described in Sections 2.2 and 5.1.

In [21]:
!awk '/^>/ { print (NR==1 ? "" : RS) $0; next } { printf "%s", $0 } END { printf RS }' SA_sample.fasta > SA_sample_temp.fasta;
!sed 's/./& /g' < SA_sample_temp.fasta > SA_sample_test.fasta;
!rm SA_sample_temp.fasta;
!Rscript SA_find_multiallelic.R SA_sample_test.fasta SA_sample_masked.fasta SA_sample_positions.fasta

[1] SNP sites: 228
[1] Plus masked: 7
[1] Plus multi-allelic: 1
[1] SNP sites:
  [1]   117   168   174   203   210   241   355   362   376   550   598  1042
 [13]  1059  1072  1172  1205  1248  1263  1269  1337  1427  1593  1912  1968
 [25]  2692  2780  2781  2782  2937  3037  3117  3182  3340  3472  3505  3904
 [37]  3923  4078  4093  4510  4615  4668  5230  5425  5495  5503  5794  5857
 [49]  5950  6525  6618  6624  6651  6701  6726  6762  7064  7113  7279  7390
 [61]  7420  7425  7844  8068  8655  8658  8660  8964  9073  9430 10138 10156
 [73] 10279 10323 10540 10623 10681 10912 11195 11230 11401 11447 11534 11629
 [85] 11653 11854 11875 11886 11896 12071 12085 12253 12503 12769 13122 13812
 [97] 14408 14583 14763 14925 14928 14937 15003 15222 15952 15970 16490 16804
[109] 17193 17334 17533 17679 17876 17898 17999 18028 18085 18175 18395 18495
[121] 18555 18788 18910 19062 19283 19542 19602 19656 20233 20268 20387 20718
[133] 21024 21099 21614 21762 21801 21979 21997 22022 22205 222

### Labelling

The sequences are given sequential labels (to make the ARGs easier to view):

In [22]:
!seqkit head -n 1 SA_sample_masked.fasta > SA_sample_masked_id.fasta;
!seqkit range -r 2:26 SA_sample_masked.fasta > f1.f;
!seqkit range -r 27:51 SA_sample_masked.fasta > f2.f;
!awk '/^>/{print ">O" ++i; next}{print}' < f1.f >> SA_sample_masked_id.fasta;
!awk '/^>/{print ">N" ++i; next}{print}' < f2.f >> SA_sample_masked_id.fasta;
!rm f1.f;
!rm f2.f

# KwARG

Detailed instructions for using KwARG and obtaining the desired outputs can be found [here](https://github.com/a-ignatieva/kwarg). KwARG is run on the masked England sample using the following command:

```
kwarg -T50,30 -Q500 -S-1,1.9,1.8,1.7,1.6,1.5,1.4,1.3,1.2,1.1,1.0,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0.01,1 -M-1,1.91,1.81,1.71,1.61,1.51,1.41,1.31,1.21,1.11,1.01,0.91,0.81,0.71,0.61,0.51,0.41,0.31,0.21,0.11,0.02,1.1 -R1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1 -C2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,-1 -k -n -f < Eng_sample_masked_id.fasta > Eng_kwarg_out.txt
```

and equivalently for the South Africa sample.